In [ ]:
!pip install numpy==1.24
!pip install --upgrade --force-reinstall gensim

In [ ]:
!pip uninstall numpy -y
!pip install numpy==1.23.5


Found existing installation: numpy 1.24.0
Uninstalling numpy-1.24.0:
  Successfully uninstalled numpy-1.24.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 73.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2025.1.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
bigframes 1.42.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
albucore 

In [ ]:
#!pip install underthesea
import pandas as pd
from underthesea import word_tokenize

# Đọc dữ liệu
df = pd.read_csv('/content/drive/MyDrive/shopee_reivew_app/train/66k_rating1.csv')

# Hàm token hóa cho tiếng Việt
def tokenize_text(text):
    # Phân tách từ bằng underthesea
    return word_tokenize(text, format="text").split()

# Tạo cột token từ bình luận (giúp các bước sau xử lý dễ dàng hơn)
df['tokens'] = df['cleaned_comment'].apply(tokenize_text)

In [ ]:

import gensim
import gensim.corpora as corpora

# Tạo dictionary và corpus từ danh sách token
dictionary = corpora.Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# Xác định số lượng chủ đề (số khía cạnh) mong muốn, ví dụ: 5
num_topics = 5

lda_model = gensim.models.LdaMulticore(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=10,
    workers=2,
    random_state=42
)

# In kết quả các chủ đề (khía cạnh)
for idx, topic in lda_model.print_topics(num_words=10):
    print("Khía cạnh {}: {}".format(idx, topic))

Khía cạnh 0: 0.042*"," + 0.033*"." + 0.032*"không" + 0.018*"có" + 0.015*"là" + 0.014*"được" + 0.012*"mình" + 0.011*"thì" + 0.009*"và" + 0.009*"mà"
Khía cạnh 1: 0.034*"." + 0.033*"," + 0.015*")" + 0.014*"-" + 0.014*"(" + 0.010*"sạc" + 0.008*"bạn" + 0.008*":" + 0.007*"và" + 0.007*"của"
Khía cạnh 2: 0.074*"," + 0.060*"hàng" + 0.049*"." + 0.042*"giao" + 0.035*"shop" + 0.034*"nhanh" + 0.026*"sản_phẩm" + 0.025*"mua" + 0.018*"chất_lượng" + 0.018*"nhận"
Khía cạnh 3: 0.122*":" + 0.078*"đúng" + 0.060*"với" + 0.054*"mô_tả" + 0.030*"sản_phẩm" + 0.029*"," + 0.029*"chất_lượng" + 0.027*"chất_liệu" + 0.025*"màu_sắc" + 0.021*"ok"
Khía cạnh 4: 0.098*":" + 0.030*"," + 0.025*"." + 0.024*"sử_dụng" + 0.016*"bao_bì" + 0.016*"/" + 0.015*"mùi" + 0.014*"đối_tượng" + 0.013*"công_dụng" + 0.011*"mẫu_mã"


In [ ]:
import torch

# Giả sử bạn đã có mô hình BERT và tokenizer được load từ file sentiment_model.pkl
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('bert_sentiment_model')
tokenizer = BertTokenizer.from_pretrained('bert_sentiment_model')

def predict_sentiment(text):
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
        sentiment = torch.argmax(outputs.logits, dim=1).item()
    return sentiment

# Ví dụ: Tách câu dựa trên dấu chấm (cần cải tiến cho tiếng Việt)
def split_into_sentences(text):
    import re
    sentences = re.split(r'[.!?]+', text)
    return [s.strip() for s in sentences if s.strip()]

# Danh sách từ khóa cho khía cạnh (ví dụ: Khía cạnh "Giá cả")
aspect_keywords = {
    "delivery": ["giao hàng", "ship", "vận chuyển", "shopee express"],
    "product": ["sản phẩm", "hỏng", "bị lỗi", "dùng", "xài", "mua", "chất lượng"],
    "packaging": ["đóng gói", "bao bì", "seal", "hộp", "móp", "rách"],
    "service": ["shop", "tư vấn", "phục vụ", "thái độ"],
    "promotion": ["khuyến mãi", "voucher", "giá", "giảm giá"]
}


def analyze_aspect_sentiment(review_text):
    results = {}
    # Tách review thành câu
    sentences = split_into_sentences(review_text)

    # Duyệt qua từng khía cạnh, kiểm tra xem câu nào chứa từ khóa của khía cạnh đó
    for aspect, keywords in aspect_keywords.items():
        aspect_sentences = [s for s in sentences if any(kw in s.lower() for kw in keywords)]
        if aspect_sentences:
            sentiments = [predict_sentiment(s) for s in aspect_sentences]
            # Tính tỷ lệ hoặc trung bình duy nhất cho khía cạnh này
            results[aspect] = {
                "sentences": aspect_sentences,
                "sentiments": sentiments  # Bạn có thể thay đổi theo cách tính tổng hợp (ví dụ: mode, trung bình cộng…)
            }
    return results

# Áp dụng cho một bình luận mẫu
review_sample = df['cleaned_comment'].iloc[0]
aspect_sentiment = analyze_aspect_sentiment(review_sample)
print(aspect_sentiment)


RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
module 'numpy' has no attribute 'dtypes'

In [ ]:
import matplotlib.pyplot as plt

# Giả sử bạn đã chạy analyze_aspect_sentiment cho tất cả các bình luận và kết hợp kết quả
# Tạo một DataFrame tổng hợp kết quả cho từng khía cạnh
aspect_summary = {
    "Giá cả": {"positive": 0, "neutral": 0, "negative": 0},
    "Chất lượng": {"positive": 0, "neutral": 0, "negative": 0},
    # Các khía cạnh khác...
}

# Ví dụ, với mỗi review trong df, bạn tính toán và cập nhật aspect_summary
for comment in df['cleaned_comment']:
    result = analyze_aspect_sentiment(comment)
    for aspect, info in result.items():
        for sentiment in info['sentiments']:
            # Giả sử: 0 - negative, 1 - neutral, 2 - positive
            if sentiment == 0:
                aspect_summary[aspect]["negative"] += 1
            elif sentiment == 1:
                aspect_summary[aspect]["neutral"] += 1
            elif sentiment == 2:
                aspect_summary[aspect]["positive"] += 1

# Sau đó trực quan hóa kết quả:
for aspect, counts in aspect_summary.items():
    labels = list(counts.keys())
    values = list(counts.values())

    plt.figure(figsize=(6,4))
    plt.bar(labels, values, color=['red', 'grey', 'green'])
    plt.title(f"Cảm xúc theo khía cạnh: {aspect}")
    plt.xlabel("Loại cảm xúc")
    plt.ylabel("Số lượng")
    plt.show()
